In [1]:
from __future__ import print_function, division
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('drive',force_remount=False)

Mounted at drive


In [53]:
class BrDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.instr = pd.read_csv(csv_file, header=0)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.instr)

    def __getitem__(self, idx):
        pc = self.instr.iloc[idx, 0:16]
        label_if_taken = self.instr.iloc[idx, 16]
        br_target = self.instr.iloc[idx, 17:33]
        h1 = self.instr.iloc[idx, 33]
        h2 = self.instr.iloc[idx, 34]
        h3 = self.instr.iloc[idx, 35]
        # nbr_instr_count = self.instr.loc[idx]["nonBranchInstrCount"]
        # trav_count = self.instr.loc[idx]["observedTraversCount"]
        if self.transform:
            return torch.tensor((pc)).unsqueeze(0), nn.functional.one_hot(torch.tensor([h1,h2,h3]), 3), label_if_taken
        else: 
        #return torch.tensor((pc, br_target, h1,h2,h3,nbr_instr_count,trav_count)), label_if_taken
            return torch.tensor((pc, br_target, h1,h2,h3)).unsqueeze(0), label_if_taken

In [ ]:
nn.functional.one_hot(torch.tensor([2,1,1]))

tensor([[0, 0, 1],
        [0, 1, 0],
        [0, 1, 0]])

In [5]:
%cd /content/drive/MyDrive/'Colab Notebooks'/CS259/parsed_traces

/content/drive/MyDrive/Colab Notebooks/CS259/parsed_traces


In [54]:
csv_file = "SHORT_SERVER-1.bt9.trace.gz0.csv"
dir = "/content/drive/MyDrive/'Colab Notebooks'/CS259/parsed_traces"
branch_dataset_train = BrDataset(csv_file, dir, True)
train_loader = DataLoader(branch_dataset_train, batch_size=1, shuffle=False)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
embedding_sizes = [(3, 2)]
# in another file:
# branch_dataset_test = BrDataset(csv_file, dir)
# test_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                          shuffle=True, num_workers=2)
'''
for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        # input is a tuple of features
        # todo: should not use float64
        inputs, label = data
        #print(inputs)
        #print(label)
'''

'\nfor epoch in range(1):  # loop over the dataset multiple times\n    running_loss = 0.0\n    for i, data in enumerate(train_loader, 0):\n        # get the inputs; data is a list of [inputs, labels]\n        # input is a tuple of features\n        # todo: should not use float64\n        inputs, label = data\n        #print(inputs)\n        #print(label)\n'

In [ ]:
for i, data in enumerate(train_loader, 0):
    inputs, h, label = data
    print(h)

In [55]:
class Test(nn.Module):
  def __init__(self, input_size, hidden_size=128, num_layers=1):
    super(Test, self).__init__()
    self.num_layers = num_layers #number of layers
    self.input_size = input_size #input size
    self.hidden_size = hidden_size #hidden state

    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True) #lstm
    self.batch1 = nn.BatchNorm1d(hidden_size)
    #self.fc_alt = nn.Linear(input_size,hidden_size)
    self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
    self.batch2 = nn.BatchNorm1d(128)
    self.fc_2 = nn.Linear(128,256)
    self.batch3 = nn.BatchNorm1d(256)
    self.fc_3 = nn.Linear(256,128)
    self.batch4 = nn.BatchNorm1d(128)
    self.fc = nn.Linear(128, 1) #fully connected last layer

    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()
    
  def forward(self,x):
    h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) #hidden state
    c_0 = torch.ones(self.num_layers, x.size(0), self.hidden_size).to(device) #internal state
    # Propagate input through LSTM
    output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
    hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
    #temp = self.fc_alt(x)
    out = self.relu(hn)
    out = self.fc_1(out) #first Dense
    out = self.relu(out) #relu
    out = self.fc_2(out) #first Dense
    out = self.relu(out) #relu
    out = self.fc_3(out) #first Dense
    out = self.relu(out) #relu
    out = self.fc(out) #Final Output
    out = self.sigmoid(out)
    out = out.float()
    return out

In [56]:
class Separated(nn.Module):
  def __init__(self, input_size, hidden_size=128, num_layers=1, embedding_sizes=embedding_sizes):
    super(Separated, self).__init__()
    self.num_layers = num_layers #number of layers
    self.input_size = input_size #input size
    self.hidden_size = hidden_size #hidden state
    
    self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories,size in embedding_sizes])
    self.emb_drop = nn.Dropout(0.6)
    self.lstm = nn.LSTM(input_size=22, hidden_size=hidden_size, num_layers=num_layers, batch_first=True) #lstm
    # self.batch0 = nn.BatchNorm1d(input_size)
    self.batch1 = nn.BatchNorm1d(hidden_size)
    #self.fc_alt = nn.Linear(input_size,hidden_size)
    self.fc_1 =  nn.Linear(22, 128) #fully connected 1
    self.batch2 = nn.BatchNorm1d(128)
    self.fc_2 = nn.Linear(128,256)
    self.batch3 = nn.BatchNorm1d(256)
    self.fc_3 = nn.Linear(256,128)
    self.batch4 = nn.BatchNorm1d(128)
    self.fc = nn.Linear(128, 1) #fully connected last layer
    self.drops = nn.Dropout(0.3)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()
    
  def forward(self, x_addr, h):
    x = [e(h[0][i]) for i,e in enumerate(self.embeddings)]

    x = torch.cat(x, 1).reshape(1, -1)

    x = self.emb_drop(x)

    # addr = self.batch0(x_addr)
    # print(x_addr, x)
    x = torch.cat([x_addr.squeeze(1), x], 1).unsqueeze(0)
    # print(x)

    h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) #hidden state
    c_0 = torch.ones(self.num_layers, x.size(0), self.hidden_size).to(device) #internal state
    # Propagate input through LSTM
    output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
    hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
    x = self.relu(self.fc_1(x))
    x = self.drops(x)
    # x = self.batch2(x)
    x = self.relu(self.fc_2(x))
    x = self.drops(x)
    # x = self.batch3(x)
    x = self.fc_3(x)
    x = self.fc(self.relu(x))
    x = self.sigmoid(x)
    return x.squeeze(0)

In [57]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(y_pred)
    # y_pred_tag = torch.round(torch.sigmoid(y_pred))
    # print(torch.sigmoid(y_pred).item(), y_pred_tag)
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [58]:
learning_rate = 1e-2
epochs = 100 # 100
input_size=16


In [59]:
model = Separated(input_size=input_size)
model = model.to(device)
# for name, para in model.named_parameters():
#     print('{}: {}'.format(name, para.shape))
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
sum = 0


In [61]:
model.train()
for e in range(1, epochs+1):
    epoch_loss = 0
    epoch_acc = 0
    #for m in model.modules():
    #  if isinstance(m, nn.LSTM):
    #    print(m.weight_hh_l0.data)
    for i, data in tqdm(enumerate(train_loader, 0)):
      inputs, h, label = data
      inputs = inputs.float()
      label = label.float()
      inputs, h, label = inputs.to(device), h.to(device), label.to(device)
      optimizer.zero_grad()
      
      y_pred = model(inputs, h)
      loss = criterion(y_pred, label.unsqueeze(1))
      acc = binary_acc(y_pred, label.unsqueeze(1))
        
      loss.backward()
      optimizer.step()
        
      epoch_loss += loss.item()
      epoch_acc += acc.item()

      if(acc != 100):
        print(y_pred.item(),end=": ")
        print(label.item(),end=", ")
    #     print(acc.item())
    print(f'\nEpoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}\n')

9it [00:00, 46.42it/s]

0.5178911089897156: 0.0, 0.5011934041976929: 0.0, 0.5132208466529846: 0.0, 0.5168732404708862: 0.0, 0.5100802779197693: 0.0, 0.5051593780517578: 0.0, 0.5215542316436768: 0.0, 

28it [00:00, 74.98it/s]

0.5176227688789368: 0.0, 0.5165815949440002: 0.0, 0.509377658367157: 0.0, 0.5231364965438843: 0.0, 0.5068836212158203: 0.0, 0.5184148550033569: 0.0, 0.5244726538658142: 0.0, 0.5214913487434387: 0.0, 

58it [00:00, 116.60it/s]

...

0.32714396715164185: 1.0, 0.24537138640880585: 1.0, 0.29718729853630066: 1.0, 

910it [00:03, 268.78it/s]

0.6833840608596802: 0.0, 3.765644578379579e-06: 1.0, 1.416638326645625e-07: 1.0, 

1023it [00:03, 263.66it/s]

1.0: 0.0, 0.0031808759085834026: 1.0, 0.5117176175117493: 0.0, 0.01589849591255188: 1.0, 
Epoch 100: | Loss: 0.51809 | Acc: 87.879



In [63]:
test_file = "SHORT_SERVER-1.bt9.trace.gz1.csv"
branch_dataset_test = BrDataset(test_file, dir, True)
test_loader = DataLoader(branch_dataset_test, batch_size=1, shuffle=False)


In [68]:
y_pred_list = []
model.eval()
test_acc = 0
with torch.no_grad():
  for i, data in enumerate(test_loader,0):
    inputs, h, label = data
    inputs = inputs.float()
    label = label.float()
    inputs, h, label = inputs.to(device), h.to(device), label.to(device)
    y_test_pred = model(inputs, h)
    y_pred_tag = torch.round(y_test_pred)
    acc = binary_acc(y_pred_tag, label.unsqueeze(1))
    test_acc += acc.item()
    y_pred_list.append(y_pred_tag.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
print(f'\nAcc: {test_acc/len(test_loader):.3f}\n')


Acc: 91.593



In [ ]:
y_pred_list